In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image
import numpy as np
import json
from skimage.draw import line
from geographiclib.geodesic import Geodesic
import math

In [2]:
# CLASS CREATED TO CONVERT WEATHER IMG TO MATRIX WITH 0, 1 (MODERATE) AND 2 (SEVERE) VALUES ONLY
class Weather():
    def __init__(self, weather_img = "weather.png" , weather_json = "weather.json"):
        
        self.weather_matrix = self.to_matrix(weather_img).astype(np.int8)
        self.bottom_right, self.top_left = self.coordinates(weather_json)
        
    def coordinates(self, weather_json):
        weather_coordinates = json.load(open('weather.json'))
        return [weather_coordinates["bottom_right"]["latitude"], weather_coordinates["bottom_right"]["longitude"]] \
                 , [ weather_coordinates["top_left"]["latitude"], weather_coordinates["top_left"]["longitude"]]
    
    def br(self):
        return self.bottom_right
    
    def tl(self):
        return self.top_left
    
    def to_matrix(self, weather_img):
        
        weather_matrix = np.array(matplotlib.image.imread(weather_img)[: ,: , 1])
        un = np.unique(weather_matrix[0], axis = 0)
        
        def map_green(x):
            return (np.isclose(x , un[1]))
        def map_yellow(x):
            return (np.isclose(x , un[2]))
        
        return weather_matrix*0 + map_green(weather_matrix)*1 + map_yellow(weather_matrix)*2
    
    def wmatrix(self):
        
        return self.weather_matrix
    
    #FUNCTION WITH RETURN NUMBER OF 0, 1 AND 2 ON THE GIVEN PATH, THE PATH IS GIVEN BY ARRAYS WITH X INDEXES AND Y INDEXES
    def weather_for_path(self, x_idx, y_idx):

        weight_of_pixels_in_line = self.weather_matrix[x_idx, y_idx]
                
        val_counts =  np.array(np.unique(weight_of_pixels_in_line, return_counts = True))
        
        weather = [0, 0, 0]
        for i in range(0, len(val_counts[0])):
            weather[val_counts[0][i]] = val_counts[1][i] 
        
        return np.array(weather)

In [3]:
# CLASS FOR FIND CORRECT DISTANCE ON EARTH SURFACE, USING LAT AND LON VALUES, ALSO CAN RETURN WAYPOINTS ON FASTEST PATH BEETWEN TWO POINTS
class Projection:
    def __init__(self, weather_img = "weather.png", weather_json_coordinates = "weather.json"):
        self.geod = Geodesic.WGS84
        self.bottom_right, self.top_left = self.coordinates(weather_json_coordinates)
        self.weather_matrix_shape = np.array(matplotlib.image.imread(weather_img)).shape
        
        self.x_grid = np.arange(self.top_left[0], self.bottom_right[0], (self.bottom_right[0]-self.top_left[0])/self.weather_matrix_shape[0])
        self.y_grid = np.arange(self.top_left[1], self.bottom_right[1], (self.bottom_right[1]-self.top_left[1])/self.weather_matrix_shape[1])
    
    
    
    def coordinates(self, weather_json):
        weather_coordinates = json.load(open('weather.json'))
        
        return [weather_coordinates["bottom_right"]["latitude"], weather_coordinates["bottom_right"]["longitude"]] \
                 , [ weather_coordinates["top_left"]["latitude"], weather_coordinates["top_left"]["longitude"]]
    
    #DISTANCE CALCULATE
    def distance(self, lat1, lon1, lat2, lon2):
        return self.geod.Inverse(lat1, lon1, lat2, lon2, Geodesic.DISTANCE)['s12']/1000
    
    #FIND WAYPOINTS IN LAT AND LON ON THE SHORTEST PATH BETWEEN TWO POINTS
    def waypoints(self, lat1, lon1, lat2, lon2, ds = 60e3):
        line = self.geod.InverseLine(lat1, lon1, lat2, lon2)
        n_steps = int(math.ceil(line.s13 / ds))
                      
        coordinates_list_lat = []
        coordinates_list_lon = []

        for i in range(n_steps + 1):
            s = min(ds * i, line.s13)
            g = line.Position(s, Geodesic.LATITUDE | Geodesic.LONGITUDE | Geodesic.LONG_UNROLL)
            coordinates_list_lat.append(g['lat2'])
            coordinates_list_lon.append(g['lon2'])
            
        return np.array(coordinates_list_lat) , np.array(coordinates_list_lon)
    
    #CONVERT WAYPOINTS IN LAT AND LON TO IDX_x AND IDX_Y IN WEATHER MATRIX
    #DS IS THE DISTANCE BETWEEN EACH WAYPOINT
    def waypoints_as_idx(self, lat1, lon1, lat2, lon2, ds = 60e3):

        waypoints_x_idx ,waypoints_y_idx = self.waypoints( lat1, lon1, lat2, lon2, ds)
        
        for i in range(0, len(waypoints_x_idx)):
            waypoints_x_idx[i] = np.abs(self.x_grid - waypoints_x_idx[i]).argmin()
            waypoints_y_idx[i] = np.abs(self.y_grid - waypoints_y_idx[i]).argmin()
        
        return waypoints_x_idx.astype(np.int16), waypoints_y_idx.astype(np.int16)
    
    #RETURN ALL PIXELS INDEXES ON THE PATH, THE PATH BETWEEN WAYPOINTS IS APPROXIMATED BY LINE
    def shortest_path_as_idx(self, lat1, lon1, lat2, lon2, ds = 60e3):
        waypoints_x_idx, waypoints_y_idx = self.waypoints_as_idx( lat1, lon1, lat2, lon2, ds)
        
        x_idx = np.array([])
        y_idx = np.array([])
        
        for i in range(0, len(waypoints_x_idx) -1 ):
            i_line = line(waypoints_x_idx[i], waypoints_y_idx[i], waypoints_x_idx[i+1], waypoints_y_idx[i+1])
            x_idx = np.append(x_idx, i_line[0][:-1])
            y_idx = np.append(y_idx, i_line[1][:-1])
        
        x_idx = np.append(x_idx, i_line[0][-1])
        y_idx = np.append(y_idx, i_line[1][-1]) 
        
        return x_idx.astype(np.int16), y_idx.astype(np.int16)

    def coordinates_to_pixel_idx(self, point):
        return  np.abs(self.x_grid - point[0]).argmin(), np.abs(self.y_grid - point[1]).argmin()